<a href="https://colab.research.google.com/github/Rashmi-debug43/Statathon/blob/main/prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv("nss.csv")
print(df.columns)
df.head()


Index(['Code', 'Title', 'Description'], dtype='object')


,Code,Title,Description
0,1111.01,State Legislatures,Is designated Title Legislators according to w...
1,1112.02,COMMISSIONER,"Diplomat DEVELOPMENT COMMISSIONER, INCOME-TAX ..."
2,1112.04,Reconciling parties in trade disputes,"Administrative Official, State inspecting scho..."
3,1112.08,Quasi-government organizations,"Establishment, etc. Regulations and by-laws of..."
4,1112.99,Departments/branches sections;,Includes officials such as: VICE Senior Govern...


In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

df["semantic_text"] = df["Title"].astype(str) + " " + df["Description"].astype(str)

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df["semantic_text"].tolist(), show_progress_bar=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

In [3]:
def semantic_search_test(text):
    q = model.encode([text])
    scores = cosine_similarity(q, embeddings)[0]
    i = int(np.argmax(scores))
    return df.iloc[i]["Title"], df.iloc[i]["Code"], round(float(scores[i]), 2)

semantic_search_test("person who manages library books")


('Attaches parts blueprints', np.float64(2621.99), 0.63)

In [4]:
import gradio as gr

def chat_backend(message, history, state):
    if history is None:
        history = []

    # FIRST MESSAGE → semantic search
    if state is None:
        title, code, conf = semantic_search_test(message)
        state = {"title": title, "code": code, "conf": conf}

        reply = (
            f"Occupation: {title}\n"
            f"Code: {code}\n"
            f"Confidence: {conf}\n\n"
            f"Ask:\n"
            f"- Why this code?\n"
            f"- How was it selected?\n"
            f"- What does confidence mean?"
        )

        history.append((message, reply))
        return history, state

    # FOLLOW‑UP QUESTIONS
    q = message.lower()

    if "why" in q:
        reply = f"The code {state['code']} corresponds to '{state['title']}'."
    elif "how" in q:
        reply = f"The system compared meanings and selected '{state['title']}'."
    elif "confidence" in q:
        reply = f"Confidence ({state['conf']}) shows match strength."
    else:
        reply = f"Mapped occupation: {state['title']} (Code: {state['code']})."

    history.append((message, reply))
    return history, state


In [5]:
with gr.Blocks() as demo:
    gr.Markdown("## Semantic Occupation Mapping – Companion Chatbot")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Enter job description or ask a question")
    state = gr.State(None)

    msg.submit(
        chat_backend,
        inputs=[msg, chatbot, state],
        outputs=[chatbot, state]
    )

demo.launch()


/tmp/ipython-input-1025676895.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
/tmp/ipython-input-1025676895.py:4: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://190fc8d4fb5c6cefe6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
